# Климатические данные

Использованы данные из архива [Pan-Arctic weather data set from publicly available in situ measurements from 1990-2023](https://erda.ku.dk/archives/e5544b25d94c11f3748555303b9a198d/published-archive.html)


In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

In [3]:
temp = pd.read_parquet('data/Combined_Russia.parquet')
temp = temp.rename(columns={'Latitude':'latitude', 'Longitude':'longitude'})
print(temp.shape)

(8142122, 40)


In [4]:
pora10_ind = pd.read_excel('data/made/pora10_ind.xlsx')

### Агрегируем данные на год с 2014 года

In [5]:
temp['year'] = temp['Datetime'].dt.year
temp['month'] = temp['Datetime'].dt.month
temp['date'] = temp['Datetime'].dt.date
temp_2014 = temp[temp['year']<=2014]
len(temp_2014)

3415521

In [6]:
temp_2014 = temp[temp['Datetime'].dt.year>=2014]
print('Процент данных c 2014 года, в которых есть:')
print(f"\n- температура воздуха - {len(temp_2014['T_air'].dropna())/len(temp_2014['T_air'])*100:.2f} %")
print(f"- осадки - {len(temp_2014['Precipitation'].dropna())/len(temp_2014['Precipitation'])*100:.2f} %")
print(f"- относительная влажность - {len(temp_2014['RH'].dropna())/len(temp_2014['RH'])*100:.2f} %")
print(f"- температура поверхности земли - {len(temp_2014['T_surf'].dropna())/len(temp_2014['T_surf'])*100:.2f} %")
print(f"- величина снежного покрова - {len(temp_2014['Snow_depth'].dropna())/len(temp_2014['Snow_depth'])*100:.2f} %")

Процент данных c 2014 года, в которых есть:

- температура воздуха - 99.67 %
- осадки - 64.43 %
- относительная влажность - 99.56 %
- температура поверхности земли - 0.17 %
- величина снежного покрова - 0.06 %


Будем работать далее с температурой воздуха `T_air`

#### Поиск ближайших измерительных станций к населенному пункту
Если расстояние от населенного пункта до ближайшей станции с измерением температуры меньше 150 км или меньше, чем минимальное расстояние + 100 км, то будем учитывать эту станцию для данного населенного пункта при расчете температур

In [7]:
dictt = {}

def to_find_near(row):
    lat0 = row['latitude']
    lon0 = row['longitude']
    name = row['settlement_name_sep']
    temp_clear = temp_2014[['latitude', 'longitude']].drop_duplicates()
    # считаем расстояние
    temp_clear['distance_km'] = round(temp_clear.apply(
        lambda row: geodesic((lat0, lon0), (row.latitude, row.longitude)).km,
        axis=1
    ), 2)
    
    temp_clear = temp_clear.sort_values(by='distance_km')
    
    dict_out = temp_clear.head(15).set_index(['latitude','longitude'])['distance_km'].to_dict()
    minn = 100000000
    dict_out_out = {}
    for key, value in dict_out.items():
        minn = min(minn, value)
        if value <= 150 or value <= minn + 100:
            dict_out_out[key] = value
    dictt[name] = dict_out_out
    
pora10_ind[['latitude','longitude', 'settlement_name_sep']].apply(to_find_near, axis=1)
len(dictt)

46

- Удаляем станции, которые не учитывается ни для одного населенного пункта
- Создаем датафрейм, в котором каждому НП будут соответствовать все рассматриваемые для него станции

In [8]:
def to_filter_stations(row):
    lat = row['latitude']
    lon = row['longitude']
    
    if (lat, lon) in lst:
        return True
    else:
        return False
def to_add_name(row):
    lat = row['latitude']
    lon = row['longitude']
    names = []
    for key, values in dictt.items():
        if (lat, lon) in values:
            names.append(key)
    return names
    
lst  = []
for i in list(dictt.values()):
    lst+=list(i.keys())
array_ = temp_2014.apply(to_filter_stations, axis = 1)
temp_2014_pora = temp_2014[array_]
temp_2014_pora['settlement_name_sep'] = temp_2014_pora.apply(to_add_name, axis = 1)
temp_2014_pora = temp_2014_pora.explode('settlement_name_sep').reset_index(drop=True)
len(temp_2014_pora)

8813730

In [9]:
len(temp_2014_pora['settlement_name_sep'].unique())

46

Усредняем данные температуры для каждого населенного пункта по каждому дню в году. Для упрощения работы с данными напишем, что это все данные за один 2024 год (заменяем все года с 2014 на 2024)

In [10]:
temp_2014_pora['date'] = pd.to_datetime(temp_2014_pora['date'])
temp_2014_pora['date_2024'] = temp_2014_pora['date'].apply(lambda dt: dt.replace(year=2024))
# temp_2014_pora['date_one_year'] = temp_2014_pora['date'].dt.strftime('%m-%d')

temp_2014_pora_tair = temp_2014_pora.dropna(subset= 'T_air')
temp_2014_pora_tair['date_2024'] = temp_2014_pora_tair['date_2024'].dt.date
year_temp = pd.DataFrame(temp_2014_pora_tair.groupby(by = ['settlement_name_sep',
                                                           'date_2024'])['T_air'].mean()).reset_index()
year_temp['date_2024'] = pd.to_datetime(year_temp['date_2024'])
year_temp = year_temp.sort_values(by = 'date_2024')

### Считаем длину отопительно периода для каждого поселения
- начало периода: более 5 дней держится средняя дневная температура ниже 8 градусов
- конец периода: более 5 дней средняя дневная температура превышает 8 градусов

In [11]:
THRESH = 8.0
RUN = 5
FULL_YEAR_DAYS = 366

df = year_temp[['settlement_name_sep', 'date_2024', 'T_air']].copy()
df['date'] = pd.to_datetime(df['date_2024'])
df['T_air'] = df['T_air'].astype(float)
df = df.sort_values(['settlement_name_sep', 'date', 'T_air'])[['settlement_name_sep', 'date', 'T_air']]
dictt = {}

for i in df['settlement_name_sep'].unique():
    dictt[i] = {}
    flag1 = 0
    flag2 = 0
    total = 0
    for _, d, t in df[df['settlement_name_sep'] == i].itertuples(index=False):
        if not flag1:
            if t>THRESH:
                total+=1
            else:
                total = 0
            if total == RUN:
                dictt[i]['spring_off_date'] = d
                flag1=1
                total = 0
        else:
            if not flag2:
                if t<THRESH:
                    total+=1
                else:
                    total = 0
                if total==RUN:
                    flag2 = 1
            else:
                dictt[i]['autumn_on_date']= d
                break
    if len(dictt[i])==0:
        dictt[i]['spring_off_date'] = pd.Timestamp(2024, 12, 30)
        dictt[i]['autumn_off_date'] = pd.Timestamp(2024, 12, 31)

res = pd.DataFrame(dictt).T.reset_index().rename(columns={'index':'settlement_name_sep'})
res['heating_days_in_year']=366-((res['autumn_on_date']-res['spring_off_date']).dt.days-1)

In [13]:
res.columns

Index(['settlement_name_sep', 'spring_off_date', 'autumn_on_date',
       'heating_days_in_year'],
      dtype='object')

In [16]:
df = pd.merge(df, res, on = 'settlement_name_sep', how = 'inner')

In [17]:
df.to_excel('data/made/year_temp.xlsx', index = False)

### Визуализируем на графике годовую динамику температур в каждом поселения с выделением отопительных периодов

### Расчеты базовых температурных характеристик за 2014-2024 гг.
- минимальная температура
- минимальная температура зимой
- максимальная температура
- максимальная температура летом
- длина отопительного периода
- амплитуда

In [187]:
temp_counts = res.copy()

temp_counts = pd.merge(round(year_temp.groupby(by = 'settlement_name_sep')['T_air'].min(), 2).reset_index().rename(columns={'T_air':'min_year_temp'}), temp_counts, on = 'settlement_name_sep', how = 'right')
temp_counts = pd.merge(round(year_temp.groupby(by = 'settlement_name_sep')['T_air'].max(), 2).reset_index().rename(columns={'T_air':'max_year_temp'}), temp_counts, on = 'settlement_name_sep', how = 'right')

year_temp['month'] = year_temp['date_2024'].dt.month
temp_counts = pd.merge(pd.DataFrame(round(year_temp[(year_temp['month']<=2)|(year_temp['month']==12)].groupby(by = 'settlement_name_sep')['T_air'].max(), 2)).reset_index().rename(columns={'T_air':'max_winter_temp'}), temp_counts, on = 'settlement_name_sep', how = 'right')
temp_counts = pd.merge(pd.DataFrame(round(year_temp[(year_temp['month']<=8)&(year_temp['month']>=6)].groupby(by = 'settlement_name_sep')['T_air'].min(), 2)).reset_index().rename(columns={'T_air':'min_summer_temp'}), temp_counts, on = 'settlement_name_sep', how = 'right')

temp_counts = pd.merge(pora10_ind[['settlement_name_sep', 'latitude', 'longitude']], temp_counts, on = 'settlement_name_sep', how = 'right')
temp_counts['amplitude'] = temp_counts['max_year_temp']-temp_counts['min_year_temp']

In [188]:
temp_counts.to_excel('data/made/temp_counts.xlsx', index = False)

In [180]:
d = pd.merge(pora10_ind, temp_counts, how = 'left', on = 'settlement_name_sep')